## Pitch Prediction SQL Queries and CSV Creation

The goal of this project is to see if I can build a model to predict what pitch a pitcher is going to throw next. As a start, I used the pitchRx package in R to create a SQLite database with all of the PITCHF/x data needed. Here I'm going to take a look at the tables in the database, write a query to get the data I need, and save it to a CSV for future use.  

In [1]:
#Import necessary modules

import pandas as pd
import numpy as np
import sqlite3

In [2]:
#Create connection to sqlite database

db_path = "2017_pitchfx.sqlite3"
con = sqlite3.connect(db_path)

In [3]:
#View tables in the database

cur = con.cursor()
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cur.fetchall()
print(tables)

[('atbat',), ('action',), ('pitch',), ('po',), ('runner',)]


In [5]:
#Print columns in tables

def print_col_names(table_list):
    for table in table_list:
        cur.execute("SELECT * FROM %s;" %table)
        names = list(map(lambda x: x[0], cur.description))
        print('Columns in %s: \n' %table, names, '\n')
        
print_col_names(tables)

Columns in atbat: 
 ['pitcher', 'batter', 'num', 'b', 's', 'o', 'start_tfs', 'start_tfs_zulu', 'stand', 'b_height', 'p_throws', 'atbat_des', 'atbat_des_es', 'event', 'score', 'home_team_runs', 'away_team_runs', 'url', 'inning_side', 'inning', 'next_', 'event2', 'event3', 'batter_name', 'pitcher_name', 'event4', 'gameday_link', 'date', 'event_num', 'event_es', 'play_guid', 'event2_es', 'end_tfs_zulu', 'event3_es'] 

Columns in action: 
 ['b', 's', 'o', 'des', 'des_es', 'event', 'tfs', 'tfs_zulu', 'player', 'pitch', 'url', 'inning_side', 'inning', 'next_', 'num', 'score', 'home_team_runs', 'away_team_runs', 'event2', 'gameday_link', 'event_es', 'event_num', 'play_guid', 'event2_es'] 

Columns in pitch: 
 ['des', 'des_es', 'id', 'type', 'tfs', 'tfs_zulu', 'x', 'y', 'sv_id', 'start_speed', 'end_speed', 'sz_top', 'sz_bot', 'pfx_x', 'pfx_z', 'px', 'pz', 'x0', 'y0', 'z0', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'break_y', 'break_angle', 'break_length', 'pitch_type', 'type_confidence', 'zone', 

## Data Required

Here is the list of columns I'm interested in for building my model:

From the atbat table:

|Column|Description|
|:-----|:-----------|
|num|Number of the at bat in the game|
|b|Balls|
|s|Strikes|
|o|Outs|
|stand|Batter handedness|
|p_throws|Pitcher handedness|
|batter_name|Batter name|
|pitcher_name|Pitcher name|
|inning_side|Top or bottom of inning|


From the pitch table:

|Column|Description|
|:-----|:-----------|
|px|the left/right distance, in feet, of the pitch from the middle of the plate as it crossed home plate. The PITCHf/x coordinate system is oriented to the catcher’s/umpire’s perspective, with distances to the right being positive and to the left being negative. ([source](https://fastballs.wordpress.com/2007/08/02/glossary-of-the-gameday-pitch-fields/))|
|pz|the height of the pitch in feet as it crossed the front of home plate. ([source](https://fastballs.wordpress.com/2007/08/02/glossary-of-the-gameday-pitch-fields/))|
|pitch_type|MLBAM's prediction for the type of pitch thrown|
|type_confidence|Level of confidence in the pitch type prediction|
|on_1b|ID of the runner on first base|
|on_2b|ID of the runner on second base|
|on_3b|ID of the runner on third base|
|count|The current count in the at bat|


Before I perform the query, I'll take a look at the tables to make sure the data looks as I expect, and to figure out what fields to join on. 

In [4]:
#Look at the top 5 columns of the 'atbat' table. 

atbat = pd.read_sql_query("SELECT * from atbat LIMIT 20", con)
atbat

,pitcher,batter,num,b,s,o,start_tfs,start_tfs_zulu,stand,b_height,...,pitcher_name,event4,gameday_link,date,event_num,event_es,play_guid,event2_es,end_tfs_zulu,event3_es
0,488984.0,434670.0,45.0,2.0,1.0,3.0,215458,2017-04-15T21:54:58Z,R,6-2,...,Tommy Hunter,None,gid_2017_04_15_tbamlb_bosmlb_1,2017_04_15,351,Roletazo de Out,c66c23f8-2210-4304-84c5-a13399ed0e46,None,None,None
1,488984.0,453064.0,57.0,2.0,3.0,3.0,012834,2017-04-07T01:28:34Z,R,6-3,...,Tommy Hunter,None,gid_2017_04_06_tormlb_tbamlb_1,2017_04_06,443,Ponche,4c0d14ab-65f5-454b-9244-e00f2b67bddd,None,2017-04-07T01:30:17Z,None
2,488984.0,434778.0,56.0,0.0,3.0,2.0,012712,2017-04-07T01:27:12Z,L,6-1,...,Tommy Hunter,None,gid_2017_04_06_tormlb_tbamlb_1,2017_04_06,435,Ponche,6322bca5-10d2-4860-9dd7-ac9e6be5ed8f,None,2017-04-07T01:28:33Z,None
3,488984.0,430832.0,67.0,1.0,2.0,2.0,200509,2017-04-09T20:05:09Z,R,6-0,...,Tommy Hunter,None,gid_2017_04_09_tormlb_tbamlb_1,2017_04_09,574,Doble,001f10f5-1c5e-48c8-b484-c9f14a7767c5,None,None,None
4,488984.0,407812.0,56.0,3.0,2.0,3.0,012739,2017-04-14T01:27:39Z,R,6-4,...,Tommy Hunter,None,gid_2017_04_13_tbamlb_nyamlb_1,2017_04_13,447,Roletazo de Out,ef27f153-6121-41c2-b0e6-612a0971fdef,None,None,None
5,488984.0,519048.0,49.0,1.0,2.0,0.0,220642,2017-04-15T22:06:42Z,L,6-2,...,Tommy Hunter,None,gid_2017_04_15_tbamlb_bosmlb_1,2017_04_15,385,Sencillo,617267e2-547a-40f2-92e9-054dc36f04c5,None,None,None
6,488984.0,455759.0,51.0,0.0,0.0,0.0,170758,2017-04-17T17:07:58Z,R,6-2,...,Tommy Hunter,None,gid_2017_04_17_tbamlb_bosmlb_1,2017_04_17,414,Sencillo,b8cfe791-1cf7-425d-a54c-858b05b401ef,None,None,None
7,488984.0,605141.0,44.0,1.0,2.0,2.0,215313,2017-04-15T21:53:13Z,R,5-9,...,Tommy Hunter,None,gid_2017_04_15_tbamlb_bosmlb_1,2017_04_15,344,Roletazo de Out,29c385e7-a2df-4d34-8f15-46ecaadde910,None,None,None
8,488984.0,593523.0,53.0,0.0,3.0,3.0,171110,2017-04-17T17:11:10Z,L,6-0,...,Tommy Hunter,None,gid_2017_04_17_tbamlb_bosmlb_1,2017_04_17,430,Ponche,3a5a2812-fac7-430c-92b7-80dd9807375d,None,None,None
9,488984.0,430832.0,55.0,3.0,2.0,1.0,012413,2017-04-07T01:24:13Z,R,6-0,...,Tommy Hunter,None,gid_2017_04_06_tormlb_tbamlb_1,2017_04_06,429,Roletazo de Out,299a3caf-3947-45bc-9570-4aa35a309e6f,None,2017-04-07T01:26:53Z,None


In [10]:
#Next let's take a look at the 'pitch' table

pitch = pd.read_sql_query("SELECT * from pitch", con)
pitch

,des,des_es,id,type,tfs,tfs_zulu,x,y,sv_id,start_speed,...,next_,num,on_1b,on_2b,on_3b,count,gameday_link,code,event_num,play_guid
0,Ball,Bola mala,3.0,B,171214,2017-04-02T17:12:14Z,78.78,203.77,170402_171214,92.1,...,Y,1.0,NaN,NaN,NaN,0-0,gid_2017_04_02_nyamlb_tbamlb_1,B,3,1a2bff45-cf05-4a65-8b08-0a5e287cd0c2
1,Called Strike,Strike cantado,4.0,S,171239,2017-04-02T17:12:39Z,111.67,159.76,170402_171240,92.4,...,Y,1.0,NaN,NaN,NaN,1-0,gid_2017_04_02_nyamlb_tbamlb_1,C,4,c4813a29-168d-4904-a12e-0a29c5503235
2,Ball,Bola mala,5.0,B,171303,2017-04-02T17:13:03Z,153.12,160.97,170402_171303,93.6,...,Y,1.0,NaN,NaN,NaN,1-1,gid_2017_04_02_nyamlb_tbamlb_1,B,5,8a0f08b0-335f-410e-b751-e1fe2e656cae
3,Called Strike,Strike cantado,6.0,S,171323,2017-04-02T17:13:23Z,148.28,162.52,170402_171323,93.2,...,Y,1.0,NaN,NaN,NaN,2-1,gid_2017_04_02_nyamlb_tbamlb_1,C,6,731141b6-a7df-4e7a-900b-fee0411f7b7b
4,Foul,Foul,7.0,S,171356,2017-04-02T17:13:56Z,87.08,194.67,170402_171356,88.0,...,Y,1.0,NaN,NaN,NaN,2-2,gid_2017_04_02_nyamlb_tbamlb_1,F,7,99ffdf74-db8e-4070-a28c-059cb25273d4
5,"In play, out(s)","En juego, out(s)",8.0,X,171429,2017-04-02T17:14:29Z,114.01,188.29,170402_171430,87.9,...,Y,1.0,NaN,NaN,NaN,2-2,gid_2017_04_02_nyamlb_tbamlb_1,X,8,f614cc42-f0f1-4b55-8954-0dda15329cd9
6,"In play, out(s)","En juego, out(s)",12.0,X,171520,2017-04-02T17:15:20Z,113.85,188.22,170402_171520,94.7,...,Y,2.0,NaN,NaN,NaN,0-0,gid_2017_04_02_nyamlb_tbamlb_1,X,12,41c737a3-dfa9-4acb-91c7-5097453ff6c7
7,Called Strike,Strike cantado,17.0,S,171632,2017-04-02T17:16:32Z,138.26,187.76,170402_171632,94.9,...,Y,3.0,NaN,NaN,NaN,0-0,gid_2017_04_02_nyamlb_tbamlb_1,C,17,4319c216-c14a-4872-a620-2e7b40990d8d
8,Ball,Bola mala,18.0,B,171649,2017-04-02T17:16:49Z,119.20,0.00,170402_171649,89.2,...,Y,3.0,NaN,NaN,NaN,0-1,gid_2017_04_02_nyamlb_tbamlb_1,B,18,7efbe046-3689-4edf-bfa1-3c358628c6a3
9,Ball,Bola mala,19.0,B,171710,2017-04-02T17:17:10Z,69.52,222.19,170402_171711,87.3,...,Y,3.0,NaN,NaN,NaN,1-1,gid_2017_04_02_nyamlb_tbamlb_1,B,19,d2dc9c60-5209-4eef-8e35-bd8c879b0268


## Performing the Query

Since I need data from two tables, I'll have to perform a join. The first column I'll use is 'gameday_link,' but I'll need to join on another column to make sure I'm capturing the correct at bats. The 'num' columns refers to the number of the at bat, and should work as the second parameter to join on.    


In [7]:
df = pd.read_sql_query("SELECT ab.pitcher_name, ab.batter_name, ab.stand, ab.p_throws, p.count, \
                       p.px, p.pz, p.pitch_type, p.on_1b, p.on_2b, p.on_3b, p.num, p.event_num, p.gameday_link \
                       FROM pitch AS p \
                       LEFT OUTER JOIN atbat AS ab \
                       ON ab.gameday_link = p.gameday_link \
                       AND ab.num = p.num;",con)

In [8]:
#Let's check to see if the query performed as expected

df.head(20)

,pitcher_name,batter_name,stand,p_throws,inning_side,count,x,y,pitch_type,type_confidence,on_1b,on_2b,on_3b,num,event_num,gameday_link
0,Christopher Archer,Brett Gardner,L,R,top,0-0,78.78,203.77,FF,2.0,NaN,NaN,NaN,1.0,3,gid_2017_04_02_nyamlb_tbamlb_1
1,Christopher Archer,Brett Gardner,L,R,top,1-0,111.67,159.76,FF,2.0,NaN,NaN,NaN,1.0,4,gid_2017_04_02_nyamlb_tbamlb_1
2,Christopher Archer,Brett Gardner,L,R,top,1-1,153.12,160.97,FF,2.0,NaN,NaN,NaN,1.0,5,gid_2017_04_02_nyamlb_tbamlb_1
3,Christopher Archer,Brett Gardner,L,R,top,2-1,148.28,162.52,FF,2.0,NaN,NaN,NaN,1.0,6,gid_2017_04_02_nyamlb_tbamlb_1
4,Christopher Archer,Brett Gardner,L,R,top,2-2,87.08,194.67,SL,2.0,NaN,NaN,NaN,1.0,7,gid_2017_04_02_nyamlb_tbamlb_1
5,Christopher Archer,Brett Gardner,L,R,top,2-2,114.01,188.29,SL,2.0,NaN,NaN,NaN,1.0,8,gid_2017_04_02_nyamlb_tbamlb_1
6,Christopher Archer,Gary Sanchez,R,R,top,0-0,113.85,188.22,FF,2.0,NaN,NaN,NaN,2.0,12,gid_2017_04_02_nyamlb_tbamlb_1
7,Christopher Archer,Gregory Bird,L,R,top,0-0,138.26,187.76,FF,2.0,NaN,NaN,NaN,3.0,17,gid_2017_04_02_nyamlb_tbamlb_1
8,Christopher Archer,Gregory Bird,L,R,top,0-1,119.20,0.00,CH,2.0,NaN,NaN,NaN,3.0,18,gid_2017_04_02_nyamlb_tbamlb_1
9,Christopher Archer,Gregory Bird,L,R,top,1-1,69.52,222.19,CH,2.0,NaN,NaN,NaN,3.0,19,gid_2017_04_02_nyamlb_tbamlb_1


To confirm that we did not omit any pitch data, we'll check the length of the pitch DataFrame vs the length of our joined DataFrame. 

In [11]:
len(df) == len(pitch)

True

Alright, looks good. Now let's explore the DataFrame and do any required cleanup, then create some new fields that'll help in the analysis. We can also close the connection to the database since we're done querying.  

In [12]:
#Close db connection
con.close()

In [13]:
#Check out the dataframe info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 725054 entries, 0 to 725053
Data columns (total 16 columns):
pitcher_name       710468 non-null object
batter_name        709521 non-null object
stand              725054 non-null object
p_throws           725054 non-null object
inning_side        725054 non-null object
count              725054 non-null object
x                  725054 non-null float64
y                  725054 non-null float64
pitch_type         719075 non-null object
type_confidence    719075 non-null float64
on_1b              220626 non-null float64
on_2b              135558 non-null float64
on_3b              68901 non-null float64
num                725054 non-null float64
event_num          725054 non-null object
gameday_link       725054 non-null object
dtypes: float64(7), object(9)
memory usage: 88.5+ MB


We see that there is some data missing in pitcher_name, batter_name, pitch_type, and the columns corresponding to men on bases. For the latter, there is a lot of data missing. Let's take a closer look.

In [14]:
df['on_1b'].value_counts()

458015.0    1329
592885.0    1037
518934.0    1027
458731.0    1021
592178.0     997
429665.0     983
592450.0     974
543333.0     971
519203.0     960
408045.0     942
502671.0     930
542303.0     911
572821.0     908
429664.0     901
446334.0     900
453568.0     897
457705.0     891
643217.0     888
594777.0     874
514917.0     874
514888.0     874
443558.0     872
553993.0     871
430832.0     869
542255.0     867
455976.0     865
541645.0     859
457763.0     856
545361.0     856
519306.0     855
            ... 
502042.0       2
502046.0       2
502706.0       2
571666.0       2
517414.0       2
594992.0       2
547179.0       2
545363.0       2
607352.0       2
595453.0       2
489232.0       2
607259.0       2
642336.0       2
543089.0       2
451705.0       2
543208.0       2
543408.0       2
628711.0       2
544928.0       2
622663.0       2
606930.0       2
533167.0       1
608650.0       1
620982.0       1
595373.0       1
607074.0       1
456124.0       1
444468.0      

It looks like these are player IDs. For now, I only really care about whether or not a man is on, not who the specific player is. Let's alter the columns to just indicate if a man is on base or not.

In [15]:
#Fill NaN's with 0s
df['on_1b'].fillna(0, inplace = True)
df['on_2b'].fillna(0, inplace = True)
df['on_3b'].fillna(0, inplace = True)

#Now convert non-0 values to 1, just to indicate there is a man on. 
df['on_1b'][df['on_1b'] > 0] = 1
df['on_2b'][df['on_2b'] > 0] = 1
df['on_3b'][df['on_3b'] > 0] = 1

#Check info
df.info()

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 725054 entries, 0 to 725053
Data columns (total 16 columns):
pitcher_name       710468 non-null object
batter_name        709521 non-null object
stand              725054 non-null object
p_throws           725054 non-null object
inning_side        725054 non-null object
count              725054 non-null object
x                  725054 non-null float64
y                  725054 non-null float64
pitch_type         719075 non-null object
type_confidence    719075 non-null float64
on_1b              725054 non-null float64
on_2b              725054 non-null float64
on_3b              725054 non-null float64
num                725054 non-null float64
event_num          725054 non-null object
gameday_link       725054 non-null object
dtypes: float64(7), object(9)
memory usage: 88.5+ MB


In [16]:
#Check to ensure all 0s and 1s
df['on_1b'].value_counts()

0.0    504428
1.0    220626
Name: on_1b, dtype: int64

Now to explore the rest of the missing data. For pitcher_name, since I want to look at specific pitchers, I don't mind dropping the rows with no data. Same with pitch_type, if I don't know the pitch type then the instance is no good for me. However, for batter_name, I don't mind not knowing the batter, so I'll fill those with UNKNOWN.

In [17]:
df['batter_name'].fillna('UNKNOWN', inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 725054 entries, 0 to 725053
Data columns (total 16 columns):
pitcher_name       710468 non-null object
batter_name        725054 non-null object
stand              725054 non-null object
p_throws           725054 non-null object
inning_side        725054 non-null object
count              725054 non-null object
x                  725054 non-null float64
y                  725054 non-null float64
pitch_type         719075 non-null object
type_confidence    719075 non-null float64
on_1b              725054 non-null float64
on_2b              725054 non-null float64
on_3b              725054 non-null float64
num                725054 non-null float64
event_num          725054 non-null object
gameday_link       725054 non-null object
dtypes: float64(7), object(9)
memory usage: 88.5+ MB


Finally, I'll drop all of the remaining NAs in the data set

In [18]:
df.dropna(inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 704701 entries, 0 to 725053
Data columns (total 16 columns):
pitcher_name       704701 non-null object
batter_name        704701 non-null object
stand              704701 non-null object
p_throws           704701 non-null object
inning_side        704701 non-null object
count              704701 non-null object
x                  704701 non-null float64
y                  704701 non-null float64
pitch_type         704701 non-null object
type_confidence    704701 non-null float64
on_1b              704701 non-null float64
on_2b              704701 non-null float64
on_3b              704701 non-null float64
num                704701 non-null float64
event_num          704701 non-null object
gameday_link       704701 non-null object
dtypes: float64(7), object(9)
memory usage: 91.4+ MB


### Feature Engineering

There are going to be some other columns that I want for my analysis. First, I'd like to split count into balls and strikes so that I can explore what pitchers are throwing in 3-x and x-2 counts. I'd also like to capture the previous pitch thrown in the at bat. 

In [21]:
#Create separate balls and strikes columns

df['b'] = df['count'].apply(lambda x: x.split('-')[0])
df['s'] = df['count'].apply(lambda x: x.split('-')[1])
df.head()

,pitcher_name,batter_name,stand,p_throws,inning_side,count,x,y,pitch_type,type_confidence,on_1b,on_2b,on_3b,num,event_num,gameday_link,b,s
0,Christopher Archer,Brett Gardner,L,R,top,0-0,78.78,203.77,FF,2.0,0.0,0.0,0.0,1.0,3,gid_2017_04_02_nyamlb_tbamlb_1,0,0
1,Christopher Archer,Brett Gardner,L,R,top,1-0,111.67,159.76,FF,2.0,0.0,0.0,0.0,1.0,4,gid_2017_04_02_nyamlb_tbamlb_1,1,0
2,Christopher Archer,Brett Gardner,L,R,top,1-1,153.12,160.97,FF,2.0,0.0,0.0,0.0,1.0,5,gid_2017_04_02_nyamlb_tbamlb_1,1,1
3,Christopher Archer,Brett Gardner,L,R,top,2-1,148.28,162.52,FF,2.0,0.0,0.0,0.0,1.0,6,gid_2017_04_02_nyamlb_tbamlb_1,2,1
4,Christopher Archer,Brett Gardner,L,R,top,2-2,87.08,194.67,SL,2.0,0.0,0.0,0.0,1.0,7,gid_2017_04_02_nyamlb_tbamlb_1,2,2


In [22]:
#Write to CSV for future use
df.to_csv('2017_pitchfx.csv', index = False)